In [1]:
from __future__ import annotations

from dotenv import load_dotenv
import os
from dataclasses import dataclass, field
from typing import List
import datetime as dt
import dateutil

from nbdev.showdoc import patch_to



In [2]:
load_dotenv('.env')

asana_client_id = os.environ['ASANA_CLIENT']
asana_client_secret = os.environ['ASANA_CLIENT_SECRET']
asana_pat = os.environ['ASANA_PAT']

asana_workspace_id = '9169352463943'

In [3]:

@dataclass
class AsanaAuth:
    token: str
    workspace_id : str
    base_url = 'https://app.asana.com/api/1.0'

    auth_header : dict = field(repr = False, default = None)

    def __post_init__(self):
        self.generate_auth_header()

    def get_auth_token(self):
        self.generate_auth_header()
        return self.token


    def generate_auth_header(self):
        self.auth_header = {
            'authorization': f'Bearer {self.token}',
        }

        return self.auth_header

In [4]:
import domolibrary.client.get_data as gd

In [5]:
def get_date(datefield):
    return dateutil.parser.parse(datefield) if datefield else None

In [6]:
@dataclass
class AsanaUser:
    id: str
    name: str
    resource_type: str

    @classmethod
    def _from_json(cls, obj):
        return cls(
            id=obj['gid'],
            name=obj['name'],
            resource_type=obj['resource_type']
        )
    
@dataclass
class AsanaProject:
    id: str
    name: str
    workspace_id:str
    permalink_url: str
    auth : AsanaAuth = field(repr = False)

    is_archived: bool
    is_completed: bool

    created_date: dt.datetime
    modified_date: dt.datetime
        
    owner: AsanaUser
    members: List[AsanaUser]
    
    due_date: dt.datetime=None
    completed_date: dt.datetime = None

    tasks : List[AsanaTask] = None

    @classmethod
    def _from_json(cls, obj, auth = None):

        owner = AsanaUser._from_json(obj.get('owner')) if obj.get('owner') else None

        members = [AsanaUser._from_json(member) for member in obj.get('members', [])] if obj.get('members') else None
        
        created_date = get_date(obj.get('created_at'))
        modified_date = get_date(obj.get('modified_at')) 
        due_date = get_date(obj.get('due_on'))
        completed_date = get_date(obj.get('completed_at'))

        return cls(
            auth = auth,
            id=obj.get('gid'),
            name=obj.get('name'),
            workspace_id=obj.get('workspace', {}).get('gid'),
            permalink_url=obj.get('permalink_url'),
            is_archived=obj.get('archived'),
            is_completed=obj.get('completed'),
            created_date=created_date,
            modified_date=modified_date,
            owner=owner,
            members=members,
            due_date=due_date,
            completed_date=completed_date
        )


In [7]:

@patch_to(AsanaProject, cls_method=True)
async def get_projects(cls : AsanaProject, auth : AsanaAuth, debug_api: bool = False, 
    debug_num_stacks_to_drop=1, return_raw : bool = False
    ):

    url = f'{auth.base_url}/projects'

    res = await gd.get_data(auth=auth, method='GET',
                             url=url,
                             num_stacks_to_drop=debug_num_stacks_to_drop,
                             debug_api=debug_api,
                             )
    if return_raw:
        return res

    return [AsanaProject._from_json(proj_obj, auth = auth) for proj_obj in res.response['data']]


In [8]:

auth = AsanaAuth(token = asana_pat, workspace_id = asana_workspace_id)

await AsanaProject.get_projects(auth, debug_api= False)


[AsanaProject(id='1202376003548112', name='Sony Workflows', workspace_id=None, permalink_url=None, is_archived=None, is_completed=None, created_date=None, modified_date=None, owner=None, members=None, due_date=None, completed_date=None, tasks=None),
 AsanaProject(id='1202585014095729', name='Domo QBR', workspace_id=None, permalink_url=None, is_archived=None, is_completed=None, created_date=None, modified_date=None, owner=None, members=None, due_date=None, completed_date=None, tasks=None),
 AsanaProject(id='1202777252066785', name='QBR Spring 2022', workspace_id=None, permalink_url=None, is_archived=None, is_completed=None, created_date=None, modified_date=None, owner=None, members=None, due_date=None, completed_date=None, tasks=None),
 AsanaProject(id='1202777252066794', name='QBR Summer 2022', workspace_id=None, permalink_url=None, is_archived=None, is_completed=None, created_date=None, modified_date=None, owner=None, members=None, due_date=None, completed_date=None, tasks=None),
 Asa

In [9]:
@patch_to(AsanaProject, cls_method=True)
async def get_by_id(cls: AsanaProject,auth : AsanaAuth, project_id, debug_api: bool = False, 
debug_num_stacks_to_drop=1, return_raw: bool = False
):
    url = f'{auth.base_url}/projects/{project_id}'

    res = await gd.get_data(auth=auth, method='GET',
                             url=url,
                             num_stacks_to_drop=debug_num_stacks_to_drop,
                             debug_api=debug_api,
                             )
    
    if return_raw: 
        return res

    return cls._from_json(res.response['data'], auth = auth)

    

In [10]:
project_id = '1202376003548112'

await AsanaProject.get_by_id(auth = auth, project_id = project_id)


AsanaProject(id='1202376003548112', name='Sony Workflows', workspace_id='9169352463943', permalink_url='https://app.asana.com/0/1202376003548112/1202376003548112', is_archived=False, is_completed=False, created_date=datetime.datetime(2022, 6, 1, 14, 1, 14, 101000, tzinfo=tzutc()), modified_date=datetime.datetime(2023, 11, 21, 16, 3, 28, 693000, tzinfo=tzutc()), owner=AsanaUser(id='1205427357991784', name='Jace McLean', resource_type='user'), members=[AsanaUser(id='1184911072633478', name='John Taft', resource_type='user'), AsanaUser(id='1202481878164303', name='Jae Myong Wilson', resource_type='user'), AsanaUser(id='1202481878164305', name='Oleksii Zakrevskyi', resource_type='user'), AsanaUser(id='1185101090774998', name='Anna Favis', resource_type='user'), AsanaUser(id='1201090018525652', name='Nate Bear', resource_type='user'), AsanaUser(id='1184911706963532', name='Brook Girma', resource_type='user'), AsanaUser(id='1185101851787805', name='Omprakash Gurubaxani', resource_type='user'

In [24]:
@dataclass
class AsanaTask:
    id: str
    name: str
    workspace_id : str

    auth : AsanaAuth = field(repr= False)
    assignee: AsanaUser = None
    
    assignee_status: str = None

    is_completed: bool = None
    
    created_at: dt.DateTime = None
    completed_on: dt.DateTime = None
    due_on: dt.DateTime = None
    modified_at: dt.DateTime = None

    memberships: List[dict] = None

    notes: str = None

    parent: Optional[dict] = None
    permalink_url: str = None
    projects: List[AsanaProject] = None

    tags: List[dict] = None

    @classmethod
    def _from_json(cls, obj: dict, auth: AsanaAuth) -> AsanaTask:
        
        assignee = AsanaUser._from_json(obj.get('assignee')) if obj.get('assignee') else None
        
        projects = [AsanaProject._from_json(project) for project in obj.get('projects', [])]
        

        return cls(
            id=obj.get('gid'),
            name=obj.get('name'),
            auth=auth,
            workspace_id=obj.get('workspace', {}).get('gid') ,
            
            assignee=assignee,

            is_completed=obj.get('completed'),
            
            assignee_status=obj.get('assignee_status'),
            
            completed_on=get_date(obj.get('completed_at')),
            created_at=get_date(obj.get('created_at')),
            due_on = get_date(obj.get('due_on')),
            modified_at=get_date(obj.get('modified_at')),
            
            
            memberships=obj.get('memberships', []),
            notes=obj.get('notes'),
            parent=obj.get('parent'),
            permalink_url=obj.get('permalink_url'),
            tags=obj.get('tags', []),

            
            projects=projects,
        )

In [30]:
@patch_to(AsanaProject)
async def get_tasks(self, debug_api: bool = False, 
    debug_num_stacks_to_drop=1, return_raw :bool = False
):
    auth = self.auth
    params = {'project' :project_id}

    url = f'{auth.base_url}/tasks'

    res = await gd.get_data(auth=auth, method='GET',
                             url=url,
                             num_stacks_to_drop=debug_num_stacks_to_drop,
                             debug_api=debug_api,
                             params=params
                             )
    
    if return_raw:
        return res

    return [AsanaTask._from_json(task_obj, auth = auth) for task_obj in res.response['data']]

    



In [35]:
project_id = '1202376003548112'

asana_project = await AsanaProject.get_by_id(project_id = project_id, auth = auth)

asana_tasks = await asana_project.get_tasks( debug_api= False)
asana_tasks[0:5]



[AsanaTask(id='1202376003548141', name='DEMO - workflow > JIRA + ServiceNow', workspace_id=None, assignee=None, assignee_status=None, is_completed=None, created_at=None, completed_on=None, due_on=None, modified_at=None, memberships=[], notes=None, parent=None, permalink_url=None, projects=[], tags=[]),
 AsanaTask(id='1205992966302553', name='DEMO - new Model, AI, API functionality', workspace_id=None, assignee=None, assignee_status=None, is_completed=None, created_at=None, completed_on=None, due_on=None, modified_at=None, memberships=[], notes=None, parent=None, permalink_url=None, projects=[], tags=[]),
 AsanaTask(id='1206010984538530', name='use of Annotations in a subscriber instance', workspace_id=None, assignee=None, assignee_status=None, is_completed=None, created_at=None, completed_on=None, due_on=None, modified_at=None, memberships=[], notes=None, parent=None, permalink_url=None, projects=[], tags=[]),
 AsanaTask(id='1206018393955400', name='Show available values in card title 

In [25]:
@patch_to(AsanaTask, cls_method=True)
async def get_by_id(cls, auth, task_id, 
    debug_api: bool = False, debug_num_stacks_to_drop=1, return_raw :bool = False
):

    url = f'{auth.base_url}/tasks/{task_id}'

    res = await gd.get_data(auth=auth, method='GET',
                             url=url,
                             num_stacks_to_drop=debug_num_stacks_to_drop,
                             debug_api=debug_api,
                             )
    
    if return_raw:
        return res

    return cls._from_json(res.response['data'], auth = auth)


In [38]:
task_id = '1205992966302553'

asana_task = await AsanaTask.get_by_id(task_id=task_id, auth= auth, return_raw= False)


from pprint import pprint
pprint(asana_task)

AsanaTask(id='1205992966302553',
          name='DEMO - new Model, AI, API functionality',
          workspace_id='9169352463943',
          assignee=AsanaUser(id='1202120071104780',
                             name='Riley Stahura',
                             resource_type='user'),
          assignee_status='inbox',
          is_completed=False,
          created_at=datetime.datetime(2023, 11, 20, 16, 22, 8, 360000, tzinfo=tzutc()),
          completed_on=None,
          due_on=datetime.datetime(2023, 11, 24, 0, 0),
          modified_at=datetime.datetime(2023, 11, 24, 7, 9, 17, 71000, tzinfo=tzutc()),
          memberships=[{'project': {'gid': '1202376003548112',
                                    'name': 'Sony Workflows',
                                    'resource_type': 'project'},
                        'section': {'gid': '1202814733264678',
                                    'name': 'Customer Use Cases + Workshop '
                                            'topics',
   

In [45]:
@dataclass
class AsanaStory:
    id: str
    created_at: dt.DateTime
    created_by: AsanaUser
    text: str
    type: str
    resource_subtype: str

    auth : AsanaAuth  = field(repr=False)

    @classmethod
    def from_json(cls, data: dict, auth : AsanaAuth) -> AsanaStory:
        created_by = AsanaUser.from_json(data['created_by']) if data.get('created_by') else None

        return cls(
            id=data.get('gid'),
            created_at = get_date(data.get('created_at')) ,
            created_by=created_by,
            text=data.get('text'),
            type=data.get('type'),
            resource_subtype=data.get('resource_subtype'),
            auth = auth
        )

In [47]:
@patch_to(AsanaTask)
async def get_stories(self, 
    debug_api: bool = False, debug_num_stacks_to_drop=1, return_raw :bool = False
):

    url = f'{auth.base_url}/tasks/{task_id}/stories'

    res = await gd.get_data(auth=auth, method='GET',
                             url=url,
                             num_stacks_to_drop=debug_num_stacks_to_drop,
                             debug_api=debug_api,
                             )
    
    if return_raw:
        return res

   return [AsanaStory._from_json(story_obj, auth = auth) for story_obj in res.response['data']]


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 17)

In [41]:
task_id = '1205992966302553'

asana_task =  await AsanaTask.get_by_id(task_id=task_id, auth= auth, return_raw= False)

res = await asana_task.get_stories(return_raw=True)

res.response['data']



[{'gid': '1205994506624641',
  'created_at': '2023-11-20T16:22:08.512Z',
  'created_by': {'gid': '1202481878164303',
   'name': 'Jae Myong Wilson',
   'resource_type': 'user'},
  'resource_type': 'story',
  'text': 'Jae Myong Wilson added to Sony Workflows',
  'type': 'system',
  'resource_subtype': 'added_to_project'},
 {'gid': '1205994111806046',
  'created_at': '2023-11-20T16:22:26.543Z',
  'created_by': {'gid': '1202481878164303',
   'name': 'Jae Myong Wilson',
   'resource_type': 'user'},
  'resource_type': 'story',
  'text': 'Jae Myong Wilson assigned to Riley Stahura',
  'type': 'system',
  'resource_subtype': 'assigned'},
 {'gid': '1205992966302559',
  'created_at': '2023-11-20T16:22:37.744Z',
  'created_by': {'gid': '1202481878164303',
   'name': 'Jae Myong Wilson',
   'resource_type': 'user'},
  'resource_type': 'story',
  'text': 'https://app.asana.com/0/1201090100584498/list is this something your team would be interested in?',
  'type': 'comment',
  'resource_subtype': 'co